In [2]:
## Install the darts library
# %pip install darts

# import neccessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from darts.utils.timeseries_generation import holidays_timeseries, TimeSeries, datetime_attribute_timeseries
from darts.dataprocessing.pipeline import Pipeline
from darts.dataprocessing.transformers import Scaler, StaticCovariatesTransformer, InvertibleMapper
from darts.models import RandomForest
from darts.metrics import mae
from darts import TimeSeries


In [ ]:
### Data Preprocessing and Feature Engineering


## Load the datasets
train = pd.read_csv("./train.csv",parse_dates=["date"])
test = pd.read_csv("./test.csv", parse_dates=["date"])
oil = pd.read_csv("./oil.csv", parse_dates=["date"]).rename(columns={"dcoilwtico":"oil"})
stores = pd.read_csv("./stores.csv")
transaction = pd.read_csv("./transactions.csv", parse_dates=["date"])

## create function to reduce the memory usage
def reduce_memory(df):
    for col in df.select_dtypes(include=["float64", "int64"]).columns:
        df[col] = pd.to_numeric(df[col], downcast="float" if df[col].dtype == "float64" else "integer")
    return df

train = reduce_memory(train)
test = reduce_memory(test)
oil = reduce_memory(oil)
stores = reduce_memory(stores)
transaction = reduce_memory(transaction)


# holiday = pd.read_csv("./holidays_events.csv", parse_dates=["date"])
date_rnge = pd.date_range(start=train["date"].min(), end=train["date"].max())
train_dt = set(train["date"])
print(date_rnge.difference(train_dt))

# set(date_rnge)-train_dt
train_dt_max = train.date.max().date()
train_dt_min = train.date.min().date()
test_dt_max = test.date.max().date()
test_dt_min = test.date.min().date()

# print(f"{train_dt_max - train_dt_min}")
print(f"Train range: {train_dt_min}:{train_dt_max}\n"
    f"Test range: {test_dt_min}:{test_dt_max}")
print(f"Date Range: {(train_dt_max - train_dt_min).days + 1}")
print(f"Actual dates: {train.date.nunique()}")

## fix the missing dates
full_date = pd.date_range(start=train["date"].min(), end=train["date"].max())

new_index = pd.MultiIndex.from_product([full_date, train["store_nbr"].unique(), train["family"].unique()],
                                       names=["date", "store_nbr","family"])

train_new = train.set_index(["date","store_nbr","family"]).reindex(new_index).reset_index()

### Fill the sales and promotion with zero values - no sales on those days
train_new[["sales", "onpromotion"]] = train_new[["sales", "onpromotion"]].fillna(0)

### fill the missing values in id column
train_new[["id"]] = pd.DataFrame(pd.RangeIndex(start=0.0, step=1, stop=len(train_new)).astype("float"))

## Oil dataset preprocessing
max_dt_oil, min_dt_oil = oil["date"].max(), oil["date"].min()
oil_dt_range = pd.DataFrame({"date":pd.date_range(start=min_dt_oil, end=max_dt_oil)})
oil_new = oil_dt_range.merge(oil, on="date", how="left").interpolate(method="linear",limit_direction="both").round(2)
oil_new["ma_7"]=oil_new["oil"].rolling(window=7, min_periods=1).mean().round(2)
oil_new["ma_14"]=oil_new["oil"].rolling(window=14, min_periods=1).mean().round(2)
oil_new["ma_21"]=oil_new["oil"].rolling(window=21, min_periods=1).mean().round(2)
oil_new["ma_28"]=oil_new["oil"].rolling(window=28, min_periods=1).mean().round(2)

## Create series
oil_series = TimeSeries.from_dataframe(oil_new, time_col="date", value_cols="oil")
oil_series_ma_7 = TimeSeries.from_dataframe(oil_new, time_col="date", value_cols="ma_7")
oil_series_ma_14 = TimeSeries.from_dataframe(oil_new, time_col="date", value_cols="ma_14")
oil_series_ma_21 = TimeSeries.from_dataframe(oil_new, time_col="date", value_cols="ma_21")
oil_series_ma_28 = TimeSeries.from_dataframe(oil_new, time_col="date", value_cols="ma_28")
oil_series = oil_series.stack(oil_series_ma_7).stack(oil_series_ma_14).stack(oil_series_ma_21).stack(oil_series_ma_28)

## create holidat series
holiday_Ecuador = holidays_timeseries(oil_series, country_code="EC")

# transaction preprocessing = handle missing values

# group the train dataset sales and and assign transaction to be 0 where sales is 0
store_sales_agg = train_new.groupby(["date", "store_nbr"]).sales.sum().reset_index().round(2)
transact = transaction.merge(store_sales_agg, on=["date", "store_nbr"], how="right")
transact.loc[transact["sales"] ==0,"transactions"] = 0
transaction = transact.drop(columns="sales").sort_values(["store_nbr","date"],ignore_index=True)

## Interpolate the remaining missing dataset
transaction["transactions"] = (transaction
                               .groupby("store_nbr", group_keys=False)
                               .transactions.apply(lambda x: x.interpolate(method="linear", 
                                                                           limit_direction="both")))

## Create transaction series
transaction_series = TimeSeries.from_group_dataframe(transaction,
                                              group_cols=["store_nbr"], 
                                              time_col="date",
                                              value_cols="transactions")


### merge the datasets

train_data = (train_new
              .merge(stores, on="store_nbr",
                     how="left").drop(columns=["id"]))


### Generate date_time attributes
day_attrb = datetime_attribute_timeseries(oil_series,attribute="day")
week_attrb = datetime_attribute_timeseries(oil_series,attribute="week")
month_attrb = datetime_attribute_timeseries(oil_series,attribute="month")
quarter_attrb = datetime_attribute_timeseries(oil_series,attribute="quarter")
year_attrb = datetime_attribute_timeseries(oil_series,attribute="year")
dayofweek_attrb = datetime_attribute_timeseries(oil_series,attribute="dayofweek")
dayofweek_cyc_attrb = datetime_attribute_timeseries(oil_series,attribute="dayofweek",cyclic=True)
dayofyear_attrb = datetime_attribute_timeseries(oil_series,attribute="dayofyear")
dayofyear_cyc_attrb = datetime_attribute_timeseries(oil_series,attribute="dayofyear", cyclic=True)
week_cyc_attrb = datetime_attribute_timeseries(oil_series,attribute="week", cyclic=True)
monthend_attrb = datetime_attribute_timeseries(oil_series,attribute="is_month_end")

## Payday Flag attributes
pay_day_flag = ((oil_series.time_index.day ==15) | (oil_series.time_index.is_month_end)).astype("int")
payday_attrb = TimeSeries.from_times_and_values(times=oil_series.time_index,values=pay_day_flag.reshape(-1,1), columns=["paydayflag"])

## Earthquake flag attributes
eart_q_start_dt = pd.to_datetime("2016-04-16")
eart_q_end_dt = eart_q_start_dt + pd.to_timedelta(4, unit="W") ## 4weeeks of impact
earth_q_flag = ((oil_series.time_index >=eart_q_start_dt) & (oil_series.time_index <= eart_q_end_dt)).astype("int")
earth_attrb = TimeSeries.from_times_and_values(times=oil_series.time_index,values=earth_q_flag.reshape(-1,1),columns=["earthquakeflag"])

## Stack all dates attributes
date_attributes = (day_attrb.stack(week_attrb).stack(month_attrb).
                   stack(quarter_attrb).stack(year_attrb)
                   .stack(dayofweek_attrb).stack(dayofweek_cyc_attrb)
                   .stack(dayofyear_attrb).stack(dayofyear_cyc_attrb)
                   .stack(week_cyc_attrb).stack(monthend_attrb)
                   .stack(payday_attrb).stack(earth_attrb))

## transaction -- past covariates
## oil, date_attributes, holiday --- future covariates
## onpromotion, city, state, type, cluster -- static covariates

## scale the transaction series using MixMax
transaction_scaled = []
for ts in transaction_series:
    scaler = Scaler()
    ts_scaled = scaler.fit_transform(ts)
    transaction_scaled.append(ts_scaled)
    
## scale the oil series using MixMax
oil_scaler = Scaler()
oil_scaled = oil_scaler.fit_transform(oil_series)

## scale the date_attributes series using MixMax
date_attribute_scaler = Scaler()
date_attribute_scaled = date_attribute_scaler.fit_transform(date_attributes)

## scale the holiday series using MixMax
holiday_scaler = Scaler()
holiday_scaled = holiday_scaler.fit_transform(holiday_Ecuador)

## future covariates
future_cov = oil_scaled.stack(date_attribute_scaled).stack(holiday_scaled)



DatetimeIndex(['2013-12-25', '2014-12-25', '2015-12-25', '2016-12-25'], dtype='datetime64[ns]', freq=None)
Train range: 2013-01-01:2017-08-15
Test range: 2017-08-16:2017-08-31
Date Range: 1688
Actual dates: 1684


In [15]:
  ## Train Global Model by Family training on a multiple series per family

forecast_horizon = 16

rf_models = {}
rf_forecasts = {}
rf_errors = {}
ts_family_scaler ={}

import gc

for family in train_data["family"].unique()[32:]:
    print(f"\nTraining Global Model for Family: {family}")
    
    # Build and scale only one family series at a time
    df = train_data[train_data["family"] == family].copy()
    series_list = TimeSeries.from_group_dataframe(df=df,
                                                  group_cols=["store_nbr"],
                                                  time_col="date",
                                                  value_cols="sales",
                                                  static_cols=["onpromotion","city","type","cluster"])

    static_scaler = StaticCovariatesTransformer()
    target_scaler_log = InvertibleMapper(fn=np.log1p, inverse_fn=np.expm1)
    target_scaler_minmax = Scaler()
    pipeline = Pipeline([static_scaler, target_scaler_log, target_scaler_minmax])
    series_list_scaled = pipeline.fit_transform(series_list)

    # Keep scalers if needed
    ts_family_scaler[family] = pipeline

    # Get past/future covariates
    past_cov_list = transaction_scaled[:len(series_list)]
    future_cov_list = [future_cov.slice(series.start_time(), series.end_time() + forecast_horizon * series.freq) for series in series_list]

    model = RandomForest(
        lags=7,
        lags_past_covariates=7,
        lags_future_covariates=(0, forecast_horizon),
        output_chunk_length=forecast_horizon,
        n_estimators=100,
        random_state=42, n_jobs=-1
    )

    model.fit(series=series_list_scaled,
              past_covariates=past_cov_list,
              future_covariates=future_cov_list)

    forecasts = model.predict(n=forecast_horizon,
                              series=series_list_scaled,
                              past_covariates=past_cov_list,
                              future_covariates=future_cov_list)

    forecasts_unscaled = pipeline.inverse_transform(forecasts)
    targets_unscaled = pipeline.inverse_transform(series_list_scaled)

    actuals = [ts[-forecast_horizon:] for ts in targets_unscaled]

    backtest = model.historical_forecasts(
        series=series_list_scaled,
        past_covariates=past_cov_list,
        future_covariates=future_cov_list,
        start=-forecast_horizon,
        forecast_horizon=forecast_horizon,
        stride=forecast_horizon,
        retrain=False,
        verbose=False
    )

    backtest_unscaled = pipeline.inverse_transform(backtest)
    mae_score = mae(actuals, backtest_unscaled)
    mean_mae = np.mean(mae_score)

    print(f"Family {family} - Final MAE: {mean_mae:.4f}")

    # Store minimal results
    rf_models[family] = model
    rf_forecasts[family] = forecasts_unscaled
    rf_errors[family] = mean_mae

    # Clear memory
    del series_list, series_list_scaled, forecasts, forecasts_unscaled, targets_unscaled
    del past_cov_list, future_cov_list, backtest, backtest_unscaled, actuals
    gc.collect()



Training Global Model for Family: SEAFOOD
Family SEAFOOD - Final MAE: 1.5372


In [ ]:
# Combine all forecast results into one DataFrame
forecast_dfs = []

for family, forecast_list in rf_forecasts.items():
    for i, ts in enumerate(forecast_list):
        store_nbr = ts.static_covariates["store_nbr"].iloc[0]
        df = ts.to_dataframe().reset_index()
        df.columns = ["date", "sales"]
        df["family"] = family
        df["store_nbr"] = store_nbr
        forecast_dfs.append(df)

# Final forecast DataFrame
forecast_df_final = pd.concat(forecast_dfs, ignore_index=True)
forecast_df_final = forecast_df_final[["date", "family", "store_nbr", "sales"]]
forecast_df_final = forecast_df_final.sort_values(by=["family", "store_nbr", "date"]).reset_index(drop=True)

# View output
print(forecast_df_final.head())


        date   family  store_nbr      sales
0 2017-08-16  SEAFOOD        1.0  21.103609
1 2017-08-17  SEAFOOD        1.0  21.539042
2 2017-08-18  SEAFOOD        1.0  21.040856
3 2017-08-19  SEAFOOD        1.0  18.032994
4 2017-08-20  SEAFOOD        1.0  14.106516


In [ ]:
## Evalute the Average Mean Absolute error of the models
key = []
val = []
for k, v in rf_errors.items():
    key.append(k)
    val.append(v)
    
df_error = pd.DataFrame({"family":key, 
                         "error":val })

print(f"The Average Mean Absolute Error is: {df_error["error"].mean()}") ## mean absolute error 22.76573340747008


In [ ]:
## Train a Global Model on individual store serie in each family


## Scale the transaction series   
transaction_scaled = {}
for ts in transaction_series:
    store_id = ts.static_covariates["store_nbr"].item() if "store_nbr" in ts.static_covariates else ts.components[0][1]
    scaler = Scaler()
    ts_scaled = scaler.fit_transform(ts)
    transaction_scaled[store_id] = ts_scaled

# set the forcast horizon
forecast_horizon = 16


# rf_models = {}
rf_forecasts = {}
rf_errors = {}
ts_series_scaler = {}

# Loop over each family first
for family in train_data["family"].unique()[32:]:
    print(f"\nProcessing Family: {family}")
    
    df_family = train_data[train_data["family"] == family].copy()
    
    # Loop over each store in this family
    for idx, store in enumerate(df_family["store_nbr"].unique()):
        print(f"  Training Model for Store: {store}")
        
        
        df = df_family[df_family["store_nbr"] == store].copy()
        
        # Convert single series dataframe to TimeSeries object
        series = TimeSeries.from_dataframe(
            df,
            time_col="date",
            value_cols="sales",
            static_covariates=df[["onpromotion","city","type","cluster"]].iloc[0]
        )
        
        # Build and fit scaler pipeline per series
        static_scaler = StaticCovariatesTransformer()
        target_scaler_log = InvertibleMapper(fn=np.log1p, inverse_fn=np.expm1)
        target_scaler_minmax = Scaler()
        pipeline = Pipeline([static_scaler, target_scaler_log, target_scaler_minmax])
        
        series_scaled = pipeline.fit_transform(series)
        
        ts_series_scaler[(family, store)] = pipeline
        
        # Prepare covariates (make sure these are prepared for this single series)
        past_cov = transaction_scaled[store]
        ## future covariates
        future_cov = oil_scaled.stack(date_attribute_scaled).stack(holiday_scaled)

        
        ## Instantiate the model
        model = RandomForest(
            lags=7,
            lags_past_covariates=7,
            lags_future_covariates=(0, forecast_horizon),
            output_chunk_length=forecast_horizon,
            n_estimators=100,
            random_state=42, n_jobs=-1
        )
        
        model.fit(
            series=series_scaled,
            past_covariates=past_cov,
            future_covariates=future_cov
        )
        
        forecasts = model.predict(
            n=forecast_horizon,
            series=series_scaled,
            past_covariates=past_cov,
            future_covariates=future_cov
        )
        
        forecasts_unscaled = pipeline.inverse_transform(forecasts)
        target_unscaled = pipeline.inverse_transform(series_scaled)
        
        actual = target_unscaled[-forecast_horizon:]
        
        ## Backtest to evaluate model performance
        backtest = model.historical_forecasts(
            series=series_scaled,
            past_covariates=past_cov,
            future_covariates=future_cov,
            start=-forecast_horizon,
            forecast_horizon=forecast_horizon,
            stride=forecast_horizon,
            retrain=False,
            verbose=False
        )
        
        backtest_unscaled = pipeline.inverse_transform(backtest)
        mae_score = mae([actual], [backtest_unscaled])
        mean_mae = np.mean(mae_score)
        
        print(f"    Store {store} - MAE: {mean_mae:.4f}")
        
        # Store results keyed by (family, store)
        # rf_models[(family, store)] = model
        rf_forecasts[(family, store)] = forecasts_unscaled
        rf_errors[(family, store)] = mean_mae
        
        # Cleanup
        del series, series_scaled, forecasts, forecasts_unscaled, target_unscaled
        del past_cov, future_cov, backtest, backtest_unscaled, actual
        gc.collect()



Processing Family: SEAFOOD
  Training Model for Store: 1
    Store 1 - MAE: 0.1968
  Training Model for Store: 10
    Store 10 - MAE: 4.6539
  Training Model for Store: 11
    Store 11 - MAE: 1.1523
  Training Model for Store: 12
    Store 12 - MAE: 0.6396
  Training Model for Store: 13
    Store 13 - MAE: 0.0232
  Training Model for Store: 14
    Store 14 - MAE: 1.4217
  Training Model for Store: 15
    Store 15 - MAE: 0.1459
  Training Model for Store: 16
    Store 16 - MAE: 0.2725
  Training Model for Store: 17
    Store 17 - MAE: 0.5112
  Training Model for Store: 18
    Store 18 - MAE: 4.1977
  Training Model for Store: 19
    Store 19 - MAE: 3.8115
  Training Model for Store: 2
    Store 2 - MAE: 0.5610
  Training Model for Store: 20
    Store 20 - MAE: 0.9530
  Training Model for Store: 21
    Store 21 - MAE: 0.1757
  Training Model for Store: 22
    Store 22 - MAE: 0.3055
  Training Model for Store: 23
    Store 23 - MAE: 1.2621
  Training Model for Store: 24
    Store 24 - MA

In [ ]:
## Fetch the forcast for all family and store_nbr

forecast_records = []

for (family, store), ts in rf_forecasts.items():
    df = ts.to_dataframe().reset_index()  
    df.columns = ['date', 'sales']
    df['family'] = family
    df['store_nbr'] = store
    forecast_records.append(df)

# Combine all forecasts into a single DataFrame
forecast_df = pd.concat(forecast_records, ignore_index=True)


In [ ]:
## Evalute the Average Mean Absolute error of the models
fam_forcast = []
str_forcast = []
error_for = []
for (family_forcast_v1, store_nbr), error_forcast_v1 in rf_errors.items():
    fam_forcast.append(family_forcast_v1)
    str_forcast.append(store_nbr)
    error_for.append(error_forcast_v1)
    
eror_df = pd.DataFrame({"family": fam_forcast,
                        "store_nbr": str_forcast,
                        "error": error_for})

print(f"The Average Mean Absolute Error is: {eror_df["error"].mean()}") ## mean absolute error 18.465658008191983

In [ ]:
## Final Submission by stacking the model

final_submission = forecast_df_final.merge(forecast_df, on=['date', 'family','store_nbr'], how="left")
final_submission["sales"] = (final_submission["sales_x"] + final_submission["sales_y"])/2
final_submission.drop(columns=["sales_x", "sales_y"], inplace=True)